## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
import tensorflow as tf
tf.enable_eager_execution()

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/aiml/prices.csv')

### Check all columns in the dataset

In [5]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [0]:
data= data.drop(["date","symbol"], axis=1)

In [8]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data["volume"]=data["volume"]/1000000

In [10]:
print(data.volume)

0          2.1636
1          2.3864
2          2.4895
3          2.0063
4          1.4086
5          1.0980
6          0.9496
7          0.7853
8          1.0937
9          1.5235
10         1.6539
11         0.9443
12         0.7449
13         0.7038
14         0.5631
15         0.8961
16         0.6804
17         0.7499
18         0.5742
19         0.6948
20         0.8963
21         0.9563
22         0.9971
23         1.2005
24         1.7252
25         1.9460
26         1.3195
27         0.9224
28         1.1851
29         0.9215
           ...   
851234     0.4642
851235     3.3432
851236     2.8241
851237     1.2218
851238    15.0955
851239     2.7075
851240     0.4582
851241     1.2306
851242     3.9803
851243     6.8720
851244     0.8112
851245     2.5389
851246     2.8253
851247     0.5246
851248     1.8885
851249     0.4661
851250     1.8876
851251     0.9592
851252     2.1117
851253     9.1178
851254     0.9492
851255    11.2504
851256     0.6462
851257     6.4316
851258    

In [11]:
data.shape

(851264, 5)

In [0]:
data_df = data.iloc[:1000]

In [13]:
data_df.count()

open      1000
close     1000
low       1000
high      1000
volume    1000
dtype: int64

In [14]:
data_df.head(5)

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
#X=data_df[['open','close','low','high']]
#Y=data_df['volume']

X=data_df.drop(['volume'],axis=1)
Y=data_df['volume']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=1)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
import numpy as np
X_train =X_train.to_numpy('float32')
X_test=X_test.to_numpy('float32')
y_train=y_train.to_numpy('float32')
y_test=y_test.to_numpy('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import normalize
#normalized_xtrain = Normalizer().fit(X_train)
#xtrain = normalized_xtrain.transform(X_train)
#normalized_xtest = Normalizer().fit(X_test)
#xtest=normalized_xtest.transform(X_test)
x_train=normalize(np.vstack([X_train]), norm='l2', axis=1)
x_test=normalize(np.vstack([X_test]), norm='l2', axis=1)



## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
W = tf.random.normal(shape=[4,1], name="Weights")
b = tf.random.normal(shape=[1],name="Bias")


2.Define a function to calculate prediction

In [0]:
def fn_prediction(x,W,b):
  y = tf.add(tf.matmul(np.array(x),W),b,name='output')
  return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def fn_loss(y, y_pred):
  loss = tf.reduce_mean(tf.square(y-y_pred),name='Loss')
  return loss

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [0]:
def fn_train(x,y, W,b,learn_rate):
  with tf.GradientTape() as t:
    t.watch([W,b])
    y_pred=fn_prediction(x_train,W,b)
    loss=fn_loss(y,y_pred)
    dc_dw,dc_db=t.gradient(loss,[W,b])
    W=W-learn_rate*dc_dw
    b=b-learn_rate*dc_db
    return W,b

### Get the shapes and values of W and b

In [25]:
for i in range(100):
  W,b = fn_train(x_train, y_train, W,b,0.01)
  print('W at:',i,'is',W)
  print('b at:',i,'is',b)
  y_pred=fn_prediction(X_train, W,b)
  train_loss=fn_loss(y_train,y_pred)
  if i%5==0:
    print('training loss at step:',i,'is',train_loss)

W at: 0 is tf.Tensor(
[[-1.4521364 ]
 [-1.1101875 ]
 [ 0.73828405]
 [ 1.8384469 ]], shape=(4, 1), dtype=float32)
b at: 0 is tf.Tensor([-0.22149089], shape=(1,), dtype=float32)
training loss at step: 0 is tf.Tensor(215.06105, shape=(), dtype=float32)
W at: 1 is tf.Tensor(
[[-1.3949246]
 [-1.0527694]
 [ 0.7949108]
 [ 1.8963537]], shape=(4, 1), dtype=float32)
b at: 1 is tf.Tensor([-0.10690248], shape=(1,), dtype=float32)
W at: 2 is tf.Tensor(
[[-1.3400012 ]
 [-0.99764794]
 [ 0.8492725 ]
 [ 1.9519444 ]], shape=(4, 1), dtype=float32)
b at: 2 is tf.Tensor([0.00310251], shape=(1,), dtype=float32)
W at: 3 is tf.Tensor(
[[-1.2872747 ]
 [-0.94473124]
 [ 0.9014598 ]
 [ 2.0053115 ]], shape=(4, 1), dtype=float32)
b at: 3 is tf.Tensor([0.10870743], shape=(1,), dtype=float32)
W at: 4 is tf.Tensor(
[[-1.2366571 ]
 [-0.89393115]
 [ 0.9515597 ]
 [ 2.0565438 ]], shape=(4, 1), dtype=float32)
b at: 4 is tf.Tensor([0.21008825], shape=(1,), dtype=float32)
W at: 5 is tf.Tensor(
[[-1.1880642 ]
 [-0.84516305]
 

### Model Prediction on 1st Examples in Test Dataset

In [26]:
for i in range(100):
  W,b = fn_train(X_test, y_test, W,b,0.01)
  print('W at:',i,'is',W)
  print('b at:',i,'is',b)
  y_pred=fn_prediction(X_test, W,b)
  train_loss=fn_loss(y_test,y_pred)
  if i%5==0:
    print('training loss at step:',i,'is',train_loss)

W at: 0 is tf.Tensor(
[[-0.05277965]
 [ 0.2941811 ]
 [ 2.1233158 ]
 [ 3.2547588 ]], shape=(4, 1), dtype=float32)
b at: 0 is tf.Tensor([2.581206], shape=(1,), dtype=float32)
training loss at step: 0 is tf.Tensor(236848.78, shape=(), dtype=float32)
W at: 1 is tf.Tensor(
[[-0.05840583]
 [ 0.2885341 ]
 [ 2.117747  ]
 [ 3.2490637 ]], shape=(4, 1), dtype=float32)
b at: 1 is tf.Tensor([2.5699368], shape=(1,), dtype=float32)
W at: 2 is tf.Tensor(
[[-0.06380696]
 [ 0.28311297]
 [ 2.112401  ]
 [ 3.2435963 ]], shape=(4, 1), dtype=float32)
b at: 2 is tf.Tensor([2.5591183], shape=(1,), dtype=float32)
W at: 3 is tf.Tensor(
[[-0.06899203]
 [ 0.27790868]
 [ 2.1072688 ]
 [ 3.2383475 ]], shape=(4, 1), dtype=float32)
b at: 3 is tf.Tensor([2.5487325], shape=(1,), dtype=float32)
W at: 4 is tf.Tensor(
[[-0.0739697 ]
 [ 0.27291256]
 [ 2.102342  ]
 [ 3.2333088 ]], shape=(4, 1), dtype=float32)
b at: 4 is tf.Tensor([2.538762], shape=(1,), dtype=float32)
W at: 5 is tf.Tensor(
[[-0.07874825]
 [ 0.26811627]
 [ 2.0

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/aiml/11_Iris.csv')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
iris=pd.get_dummies(iris_df,columns=["Species"])

In [0]:
# load dataset

X = iris.iloc[:,1:5]
Y = iris.iloc[:,5:8]

In [0]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense

from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

### Splitting the data into feature set and target set

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=1)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:

	# create model
	model = tf.keras.Sequential()
	#model.add(tf.keras.layers.Dense(8, input_dim=4, activation='relu'))
	model.add(tf.keras.layers.Dense(3, input_shape=(4,), activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
 
	

### Model Training 

In [96]:
 model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=150, batch_size=10)

Train on 105 samples, validate on 45 samples
Epoch 1/150
105/105 [==============================] - 0s 2ms/sample - loss: 4.8043 - acc: 0.3048 - val_loss: 3.4033 - val_acc: 0.4222
Epoch 2/150
105/105 [==============================] - 0s 403us/sample - loss: 3.1935 - acc: 0.4000 - val_loss: 2.7215 - val_acc: 0.2889
Epoch 3/150
105/105 [==============================] - 0s 406us/sample - loss: 2.4339 - acc: 0.3714 - val_loss: 1.9658 - val_acc: 0.5556
Epoch 4/150
105/105 [==============================] - 0s 414us/sample - loss: 1.7264 - acc: 0.3524 - val_loss: 1.3395 - val_acc: 0.2889
Epoch 5/150
105/105 [==============================] - 0s 395us/sample - loss: 1.2049 - acc: 0.3714 - val_loss: 1.1280 - val_acc: 0.2889
Epoch 6/150
105/105 [==============================] - 0s 381us/sample - loss: 1.0629 - acc: 0.3619 - val_loss: 1.0426 - val_acc: 0.2889
Epoch 7/150
105/105 [==============================] - 0s 411us/sample - loss: 0.9870 - acc: 0.5429 - val_loss: 0.9843 - val_acc: 0.422

### Model Prediction

In [97]:
# evaluate the model
scores = model.evaluate(X_test, y_test)
scores[1]*100

45/45 [==============================] - 0s 1ms/sample - loss: 0.3513 - acc: 0.8667


86.66666746139526

In [0]:

y_pred= model.predict_classes(X_test)


### Save the Model

In [0]:
model.save('Keras_')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?